#Data Cleaning  
##### **Extract the views of the wikipedia articles for 2018-19 from the JSON files and convert it to a dataframe**

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import re
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
!pip install pageviewapi
import pageviewapi

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Mounting the Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Setting the directory paths**

In [ ]:
baseDir = '/content/drive/MyDrive/Time Series Analysis (Mini project)/Time Series Analysis (Mini project)'
datasets = 'Datasets'
datapath1 = 'data_2015-17.csv'
datapath2 = 'json_files'

**Adding the directory containing the custom modules**

In [ ]:
import sys
sys.path.append(baseDir + '/utils')
# Custom module to fill in the views if not already requested
# import dataFill.py

**Importing the dataset**

In [ ]:
train_data = pd.read_csv(os.path.join(baseDir, datasets, datapath1))
train_data.head(10)

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,...,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,19.0,10.0,14.0,15.0,8.0,16.0,8.0,8.0,16.0,7.0,11.0,10.0,20.0,18.0,15.0,14.0,49.0,10.0,16.0,18.0,8.0,5.0,9.0,7.0,13.0,9.0,7.0,4.0,11.0,...,46.0,24.0,22.0,22.0,11.0,25.0,45.0,39.0,17.0,18.0,13.0,57.0,18.0,14.0,19.0,39.0,22.0,16.0,28.0,21.0,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,41.0,65.0,57.0,38.0,20.0,62.0,44.0,15.0,10.0,47.0,24.0,17.0,22.0,9.0,39.0,13.0,11.0,12.0,21.0,19.0,9.0,15.0,33.0,8.0,8.0,7.0,13.0,2.0,23.0,...,39.0,20.0,31.0,19.0,7.0,11.0,20.0,13.0,16.0,16.0,47.0,21.0,20.0,12.0,13.0,20.0,22.0,20.0,12.0,37.0,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,1.0,1.0,1.0,6.0,8.0,6.0,4.0,5.0,1.0,2.0,3.0,8.0,8.0,6.0,6.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,5.0,3.0,5.0,4.0,2.0,5.0,1.0,...,22.0,8.0,4.0,3.0,1.0,3.0,7.0,3.0,1.0,9.0,4.0,2.0,6.0,3.0,3.0,3.0,8.0,2.0,4.0,3.0,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,16.0,11.0,23.0,145.0,14.0,17.0,85.0,4.0,30.0,22.0,9.0,10.0,11.0,7.0,7.0,11.0,9.0,11.0,44.0,8.0,14.0,19.0,10.0,17.0,17.0,10.0,7.0,10.0,1.0,...,37.0,14.0,12.0,13.0,11.0,10.0,14.0,15.0,15.0,11.0,19.0,9.0,19.0,8.0,17.0,11.0,18.0,20.0,11.0,11.0,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,39.0,66.0,114.0,19.0,29.0,12.0,11.0,14.0,14.0,63.0,12.0,22.0,14.0,20.0,11.0,23.0,66.0,19.0,23.0,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0
5,5566_zh.wikipedia.org_all-access_spider,12.0,7.0,4.0,5.0,20.0,8.0,5.0,17.0,24.0,7.0,12.0,11.0,7.0,9.0,6.0,10.0,8.0,13.0,3.0,14.0,4.0,9.0,14.0,10.0,8.0,3.0,74.0,17.0,8.0,6.0,9.0,3.0,10.0,21.0,9.0,5.0,3.0,4.0,1.0,...,41.0,35.0,18.0,18.0,15.0,16.0,13.0,14.0,29.0,18.0,28.0,21.0,10.0,19.0,13.0,8.0,54.0,19.0,14.0,41.0,32.0,46.0,31.0,29.0,31.0,28.0,17.0,16.0,8.0,13.0,13.0,13.0,45.0,4.0,13.0,20.0,18.0,17.0,14.0,11.0
6,91Days_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,33.0,8.0,7.0,5.0,12.0,7.0,9.0,11.0,8.0,10.0,10.0,7.0,51.0,12.0,10.0,14.0,11.0,9.0,9.0,9.0,68.0,8.0,40.0,6.0,14.0,11.0,6.0,9.0,7.0,12.0,8.0,5.0,7.0,8.0,10.0,8.0,5.0,3.0,5.0
7,A'N'D_zh.wikipedia.org_all-access_spider,118.0,26.0,30.0,24.0,29.0,127.0,53.0,37.0,20.0,32.0,17.0,23.0,47.0,33.0,47.0,58.0,29.0,187.0,128.0,34.0,38.0,8.0,38.0,17.0,45.0,14.0,15.0,56.0,30.0,15.0,115.0,6.0,25.0,10.0,135.0,40.0,63.0,32.0,35.0,...,43.0,33.0,20.0,

**Checking for null values**

In [ ]:
train_data.isnull().sum()

Page              0
2015-07-01    20740
2015-07-02    20816
2015-07-03    20544
2015-07-04    20654
              ...  
2017-09-06     1775
2017-09-07     3467
2017-09-08     1061
2017-09-09     3332
2017-09-10     5578
Length: 804, dtype: int64

In [ ]:
missingData = train_data[train_data.isnull().any(axis=1)]
missingData.iloc[np.r_[0:10, len(missingData)-10:len(missingData)]]

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,...,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,39.0,66.0,114.0,19.0,29.0,12.0,11.0,14.0,14.0,63.0,12.0,22.0,14.0,20.0,11.0,23.0,66.0,19.0,23.0,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0
6,91Days_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,33.0,8.0,7.0,5.0,12.0,7.0,9.0,11.0,8.0,10.0,10.0,7.0,51.0,12.0,10.0,14.0,11.0,9.0,9.0,9.0,68.0,8.0,40.0,6.0,14.0,11.0,6.0,9.0,7.0,12.0,8.0,5.0,7.0,8.0,10.0,8.0,5.0,3.0,5.0
10,ASTRO_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,1.0,0.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,62.0,12.0,29.0,34.0,34.0,14.0,29.0,59.0,25.0,25.0,18.0,13.0,20.0,28.0,28.0,28.0,13.0,9.0,15.0,15.0,38.0,23.0,51.0,20.0,36.0,25.0,11.0,12.0,19.0,17.0,17.0,25.0,4.0,29.0,14.0,44.0,62.0,47.0,16.0
13,AlphaGo_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,35.0,36.0,18.0,17.0,17.0,29.0,20.0,59.0,23.0,30.0,20.0,26.0,28.0,30.0,24.0,23.0,22.0,28.0,18.0,20.0,41.0,26.0,17.0,21.0,28.0,20.0,17.0,18.0,26.0,21.0,16.0,25.0,15.0,19.0,25.0,13.0,34.0,26.0,14.0
19,B-PROJECT_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.0,9.0,13.0,10.0,3.0,9.0,6.0,6.0,5.0,6.0,11.0,8.0,12.0,28.0,3.0,5.0,13.0,8.0,10.0,15.0,10.0,25.0,13.0,14.0,12.0,12.0,17.0,12.0,16.0,15.0,12.0,13.0,9.0,4.0,4.0,15.0,7.0,10.0,4.0,6.0
24,BLACK_PINK_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.0,29.0,20.0,21.0,25.0,9.0,24.0,18.0,19.0,15.0,21.0,10.0,18.0,21.0,17.0,10.0,21.0,32.0,27.0,22.0,17.0,45.0,30.0,26.0,18.0,25.0,28.0,21.0,25.0,23.0,24.0,23.0,23.0,22.0,20.0,17.0,14.0,15.0,14.0,12.0
27,Beautiful_Mind_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.0,8.0,5.0,7.0,8.0,6.0,9.0,6.0,6.0,2.0,8.0,5.0,6.0,8.0,3.0,13.0,14.0,7.0,14.0,8.0,11.0,30.0,14.0,15.0,12.0,17.0,11.0,14.0,7.0,12.0,8.0,10.0,11.0,10.0,6.0,6.0,9.0,16.0,7.0,9.0
37,Dear_My_Friends_zh.wikipedia.org_all-access_sp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,13.0,7.0,17.0,13.0,12.0,9.0,14.0,16.0,12.0,7.0,9.0,15.0,11.0,14.0,12.0,17.0,10.0,5.0,18.0,10.0,38.0,19.0,15.0,13.0,

**Handling the missing values**

In [ ]:
train_data = train_data.fillna(0)
train_data.isnull().sum()

Page          0
2015-07-01    0
2015-07-02    0
2015-07-03    0
2015-07-04    0
             ..
2017-09-06    0
2017-09-07    0
2017-09-08    0
2017-09-09    0
2017-09-10    0
Length: 804, dtype: int64

In [ ]:
train_data.head(10)

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,...,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,19.0,10.0,14.0,15.0,8.0,16.0,8.0,8.0,16.0,7.0,11.0,10.0,20.0,18.0,15.0,14.0,49.0,10.0,16.0,18.0,8.0,5.0,9.0,7.0,13.0,9.0,7.0,4.0,11.0,...,46.0,24.0,22.0,22.0,11.0,25.0,45.0,39.0,17.0,18.0,13.0,57.0,18.0,14.0,19.0,39.0,22.0,16.0,28.0,21.0,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,41.0,65.0,57.0,38.0,20.0,62.0,44.0,15.0,10.0,47.0,24.0,17.0,22.0,9.0,39.0,13.0,11.0,12.0,21.0,19.0,9.0,15.0,33.0,8.0,8.0,7.0,13.0,2.0,23.0,...,39.0,20.0,31.0,19.0,7.0,11.0,20.0,13.0,16.0,16.0,47.0,21.0,20.0,12.0,13.0,20.0,22.0,20.0,12.0,37.0,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,1.0,1.0,1.0,6.0,8.0,6.0,4.0,5.0,1.0,2.0,3.0,8.0,8.0,6.0,6.0,2.0,2.0,3.0,2.0,4.0,3.0,3.0,5.0,3.0,5.0,4.0,2.0,5.0,1.0,...,22.0,8.0,4.0,3.0,1.0,3.0,7.0,3.0,1.0,9.0,4.0,2.0,6.0,3.0,3.0,3.0,8.0,2.0,4.0,3.0,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,16.0,11.0,23.0,145.0,14.0,17.0,85.0,4.0,30.0,22.0,9.0,10.0,11.0,7.0,7.0,11.0,9.0,11.0,44.0,8.0,14.0,19.0,10.0,17.0,17.0,10.0,7.0,10.0,1.0,...,37.0,14.0,12.0,13.0,11.0,10.0,14.0,15.0,15.0,11.0,19.0,9.0,19.0,8.0,17.0,11.0,18.0,20.0,11.0,11.0,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,23.5,10.0,7.0,49.5,12.0,17.0,9.5,13.0,17.5,11.5,14.0,11.0,15.0,77.0,10.0,13.5,46.5,8.5,16.5,18.0,6.5,9.5,12.5,8.5,7.5,7.0,41.5,14.0,26.0,7.0,11.5,11.0,10.0,19.0,13.0,7.5,5.0,7.0,1.0,...,40.0,39.0,66.0,114.0,19.0,29.0,12.0,11.0,14.0,14.0,63.0,12.0,22.0,14.0,20.0,11.0,23.0,66.0,19.0,23.0,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0
5,5566_zh.wikipedia.org_all-access_spider,12.0,7.0,4.0,5.0,20.0,8.0,5.0,17.0,24.0,7.0,12.0,11.0,7.0,9.0,6.0,10.0,8.0,13.0,3.0,14.0,4.0,9.0,14.0,10.0,8.0,3.0,74.0,17.0,8.0,6.0,9.0,3.0,10.0,21.0,9.0,5.0,3.0,4.0,1.0,...,41.0,35.0,18.0,18.0,15.0,16.0,13.0,14.0,29.0,18.0,28.0,21.0,10.0,19.0,13.0,8.0,54.0,19.0,14.0,41.0,32.0,46.0,31.0,29.0,31.0,28.0,17.0,16.0,8.0,13.0,13.0,13.0,45.0,4.0,13.0,20.0,18.0,17.0,14.0,11.0
6,91Days_zh.wikipedia.org_all-access_spider,65.0,16.5,17.0,14.5,24.5,67.5,29.0,27.0,22.0,19.5,14.5,17.0,27.0,21.0,26.5,34.0,18.5,100.0,65.5,24.0,21.0,8.5,26.0,13.5,26.5,8.5,44.5,36.5,19.0,10.5,62.0,4.5,17.5,15.5,72.0,22.5,33.0,18.0,18.0,...,29.0,33.0,8.0,7.0,5.0,12.0,7.0,9.0,11.0,8.0,10.0,10.0,7.0,51.0,12.0,10.0,14.0,11.0,9.0,9.0,9.0,68.0,8.0,40.0,6.0,14.0,11.0,6.0,9.0,7.0,12.0,8.0,5.0,7.0,8.0,10.0,8.0,5.0,3.0,5.0
7,A'N'D_zh.wikipedia.org_all-access_spider,118.0,26.0,30.0,24.0,29.0,127.0,53.0,37.0,20.0,32.0,17.0,23.0,47.0,33.0,47.0,58.0,29.0,187.0,128.0,34.0,38.0,8.0,38.0,17.0,45.0,14.0,15.0,56.0,30.0,15.0,11

In [ ]:
train_data.to_csv(os.path.join(baseDir, datasets, 'train_data.csv'), index = False)

# **Creating the Test Dataframe**

In [ ]:
path = os.path.join(baseDir, datasets, datapath2)
testDataframe = pd.DataFrame(data = [], columns= ['Page'])
testDataframe

,Page


**Extracting all the files that were downloaded by using pageview API**

In [ ]:
JSONFiles = os.listdir(path)
len(JSONFiles)

86461

In [ ]:
JSONFiles = pd.DataFrame(JSONFiles, columns=['Page'])

In [ ]:
# JSONFiles = pd.read_csv(os.path.join(baseDir, datasets, 'test_dataset.csv'))
# JSONFiles

**Creating the test dataframe, by extracting views of all articles over a period of time**

In [ ]:
count = 0
for _ in range(len(list(JSONFiles['Page']))):
  with open (os.path.join(path,list(JSONFiles['Page'])[_])) as testFile:
    views = json.load(testFile)
    if views != '':
      for _ in range(len(views['items'])):
        testDataframe.at[views['items'][_]['article'] + '_' + views['items'][_]['project'] + '_' + views['items'][_]['access'] + '_' + views['items'][_]['agent'], views['items'][_]['timestamp']] = views['items'][_]['views']
  count += 1
  print(count)
testDataframe.drop(columns = ['Page'], inplace = True)
testDataframe.index.name = 'Page'
testDataframe.reset_index()

Streaming output truncated to the last 5000 lines.
64024
64025
64026
64027
64028
64029
64030
64031
64032
64033
64034
64035
64036
64037
64038
64039
64040
64041
64042
64043
64044
64045
64046
64047
64048
64049
64050
64051
64052
64053
64054
64055
64056
64057
64058
64059
64060
64061
64062
64063
64064
64065
64066
64067
64068
64069
64070
64071
64072
64073
64074
64075
64076
64077
64078
64079
64080
64081
64082
64083
64084
64085
64086
64087
64088
64089
64090
64091
64092
64093
64094
64095
64096
64097
64098
64099
64100
64101
64102
64103
64104
64105
64106
64107
64108
64109
64110
64111
64112
64113
64114
64115
64116
64117
64118
64119
64120
64121
64122
64123
64124
64125
64126
64127
64128
64129
64130
64131
64132
64133
64134
64135
64136
64137
64138
64139
64140
64141
64142
64143
64144
64145
64146
64147
64148
64149
64150
64151
64152
64153
64154
64155
64156
64157
64158
64159
64160
64161
64162
64163
64164
64165
64166
64167
64168
64169
64170
64171
64172
64173
64174
64175
64176
64177
64178
64179
64180
64181
6

JSONDecodeError: ignored

**Save the test data in a CSV file for later use**

In [ ]:
# testDataframe.drop(columns = ['Page'], inplace = True)
# testDataframe.index.name = 'Page'
# testDataframe.reset_index()
testDataframe.to_csv(os.path.join(baseDir, datasets, 'test_data.csv'))

In [ ]:
testData = pd.read_csv(os.path.join(baseDir, datasets, 'test_data.csv'))
testData

,Unnamed: 0,Page,2018010100,2018010200,2018010300,2018010400,2018010500,2018010600,2018010700,2018010800,2018010900,2018011000,2018011100,2018011200,2018011300,2018011400,2018011500,2018011600,2018011700,2018011800,2018011900,2018012000,2018012100,2018012200,2018012300,2018012400,2018012500,2018012600,2018012700,2018012800,2018012900,2018013000,2018013100,2018020100,2018020200,2018020300,2018020400,2018020500,2018020600,2018020700,...,2019112200,2019112300,2019112400,2019112500,2019112600,2019112700,2019112800,2019112900,2019113000,2019120100,2019120200,2019120300,2019120400,2019120500,2019120600,2019120700,2019120800,2019120900,2019121000,2019121100,2019121200,2019121300,2019121400,2019121500,2019121600,2019121700,2019121800,2019121900,2019122000,2019122100,2019122200,2019122300,2019122400,2019122500,2019122600,2019122700,2019122800,2019122900,2019123000,2019123100
0,黒い霧事件_(日本プロ野球)_ja.wikipedia_desktop_all-agents,NaN,154.0,128.0,160.0,208.0,322.0,218.0,168.0,353.0,248.0,201.0,119.0,169.0,115.0,108.0,220.0,171.0,154.0,151.0,232.0,106.0,253.0,677.0,231.0,249.0,521.0,184.0,247.0,192.0,207.0,340.0,190.0,166.0,393.0,237.0,263.0,330.0,189.0,275.0,...,121.0,52.0,99.0,592.0,252.0,279.0,711.0,236.0,95.0,129.0,144.0,386.0,880.0,581.0,358.0,159.0,125.0,311.0,178.0,185.0,124.0,143.0,62.0,107.0,195.0,140.0,116.0,97.0,114.0,88.0,83.0,134.0,135.0,173.0,116.0,137.0,82.0,80.0,80.0,87.0
1,笹川陽平_ja.wikipedia_desktop_all-agents,NaN,38.0,77.0,78.0,65.0,75.0,51.0,63.0,64.0,87.0,91.0,115.0,86.0,52.0,59.0,75.0,63.0,108.0,94.0,70.0,65.0,62.0,91.0,67.0,97.0,92.0,93.0,71.0,58.0,104.0,118.0,109.0,80.0,84.0,57.0,89.0,125.0,160.0,95.0,...,61.0,56.0,50.0,76.0,70.0,187.0,118.0,125.0,49.0,60.0,95.0,86.0,91.0,91.0,97.0,74.0,52.0,152.0,194.0,172.0,87.0,72.0,50.0,39.0,82.0,74.0,82.0,110.0,75.0,72.0,52.0,118.0,105.0,125.0,100.0,404.0,54.0,45.0,51.0,56.0
2,バンジージャンプ_ja.wikipedia_desktop_all-agents,NaN,21.0,16.0,19.0,26.0,35.0,48.0,87.0,54.0,33.0,38.0,38.0,22.0,28.0,16.0,26.0,38.0,16.0,29.0,23.0,38.0,21.0,26.0,25.0,13.0,23.0,21.0,31.0,11.0,48.0,19.0,31.0,26.0,23.0,17.0,27.0,41.0,34.0,25.0,...,29.0,30.0,31.0,31.0,46.0,29.0,38.0,45.0,32.0,35.0,26.0,45.0,36.0,36.0,30.0,30.0,25.0,31.0,44.0,39.0,39.0,29.0,15.0,24.0,41.0,30.0,23.0,25.0,40.0,26.0,26.0,27.0,40.0,25.0,31.0,31.0,21.0,33.0,20.0,22.0
3,バック・トゥ・ザ・フューチャー_ja.wikipedia_desktop_all-agents,NaN,232.0,306.0,340.0,318.0,271.0,292.0,396.0,386.0,341.0,374.0,358.0,329.0,316.0,346.0,341.0,428.0,365.0,377.0,341.0,324.0,320.0,392.0,412.0,286.0,353.0,349.0,326.0,366.0,358.0,418.0,363.0,365.0,384.0,383.0,377.0,361.0,367.0,355.0,...,467.0,417.0,442.0,425.0,394.0,475.0,344.0,528.0,399.0,522.0,435.0,408.0,378.0,428.0,401.0,414.0,404.0,370.0,431.0,349.0,489.0,492.0,417.0,359.0,420.0,436.0,418.0,482.0,423.0,1083.0,527.0,592.0,497.0,470.0,502.0,535.0,478.0,502.0,2007.0,1336.0
4,メーガン・マークル_ja.wikipedia_desktop_all-agents,NaN,199.0,128.0,197.0,164.0,200.0,200.0,603.0,327.0,301.0,215.0,234.0,1200.0,499.0,263.0,439.0,758.0,404.0,641.0,512.0,162.0,203.0,259.0,175.0,173.0,155.0,137.0,117.0,151.0,242.0,122.0,142.0,136.0,177.0,125.0,90.0,105.0,151.0,164.0,...,7.0,6.0,8.0,7.0,10.0,8.0,8.0,16.0,10.0,8.0,13.0,12.0,7.0,11.0,8.0,11.0,12.0,11.0,3.0,6.0,8.0,6.0,8.0,8.0,8.0,13.0,5.0,5.0,5.0,11.0,5.0,7.0,5.0,7.0,8.0,10.0,9.0,7.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63851,User:Kingchang711_zh.wikipedia_all-access_spider,NaN,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,4.0,0.0,1.0,NaN,1.0,11.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,2.0,1.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,2.0,NaN,...,NaN,1.0,3.0,1.0,NaN,2.0,NaN,NaN,1.0,0.0,0.0,1.0,3.0,1.0,3.0,NaN,3.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN,1.0,NaN,3.0,NaN,1.0,1.0,1.0,NaN,NaN,0.0,NaN,NaN,5.0,3.0,1.0,NaN
63852,User:Caoliucm_zh.wi

In [ ]:
testData.drop(columns = ['Page'], inplace = True)
testData.rename(columns={'Unnamed: 0': 'Page'}, inplace = True)
testData

,Page,2018010100,2018010200,2018010300,2018010400,2018010500,2018010600,2018010700,2018010800,2018010900,2018011000,2018011100,2018011200,2018011300,2018011400,2018011500,2018011600,2018011700,2018011800,2018011900,2018012000,2018012100,2018012200,2018012300,2018012400,2018012500,2018012600,2018012700,2018012800,2018012900,2018013000,2018013100,2018020100,2018020200,2018020300,2018020400,2018020500,2018020600,2018020700,2018020800,...,2019112200,2019112300,2019112400,2019112500,2019112600,2019112700,2019112800,2019112900,2019113000,2019120100,2019120200,2019120300,2019120400,2019120500,2019120600,2019120700,2019120800,2019120900,2019121000,2019121100,2019121200,2019121300,2019121400,2019121500,2019121600,2019121700,2019121800,2019121900,2019122000,2019122100,2019122200,2019122300,2019122400,2019122500,2019122600,2019122700,2019122800,2019122900,2019123000,2019123100
0,黒い霧事件_(日本プロ野球)_ja.wikipedia_desktop_all-agents,154.0,128.0,160.0,208.0,322.0,218.0,168.0,353.0,248.0,201.0,119.0,169.0,115.0,108.0,220.0,171.0,154.0,151.0,232.0,106.0,253.0,677.0,231.0,249.0,521.0,184.0,247.0,192.0,207.0,340.0,190.0,166.0,393.0,237.0,263.0,330.0,189.0,275.0,313.0,...,121.0,52.0,99.0,592.0,252.0,279.0,711.0,236.0,95.0,129.0,144.0,386.0,880.0,581.0,358.0,159.0,125.0,311.0,178.0,185.0,124.0,143.0,62.0,107.0,195.0,140.0,116.0,97.0,114.0,88.0,83.0,134.0,135.0,173.0,116.0,137.0,82.0,80.0,80.0,87.0
1,笹川陽平_ja.wikipedia_desktop_all-agents,38.0,77.0,78.0,65.0,75.0,51.0,63.0,64.0,87.0,91.0,115.0,86.0,52.0,59.0,75.0,63.0,108.0,94.0,70.0,65.0,62.0,91.0,67.0,97.0,92.0,93.0,71.0,58.0,104.0,118.0,109.0,80.0,84.0,57.0,89.0,125.0,160.0,95.0,102.0,...,61.0,56.0,50.0,76.0,70.0,187.0,118.0,125.0,49.0,60.0,95.0,86.0,91.0,91.0,97.0,74.0,52.0,152.0,194.0,172.0,87.0,72.0,50.0,39.0,82.0,74.0,82.0,110.0,75.0,72.0,52.0,118.0,105.0,125.0,100.0,404.0,54.0,45.0,51.0,56.0
2,バンジージャンプ_ja.wikipedia_desktop_all-agents,21.0,16.0,19.0,26.0,35.0,48.0,87.0,54.0,33.0,38.0,38.0,22.0,28.0,16.0,26.0,38.0,16.0,29.0,23.0,38.0,21.0,26.0,25.0,13.0,23.0,21.0,31.0,11.0,48.0,19.0,31.0,26.0,23.0,17.0,27.0,41.0,34.0,25.0,27.0,...,29.0,30.0,31.0,31.0,46.0,29.0,38.0,45.0,32.0,35.0,26.0,45.0,36.0,36.0,30.0,30.0,25.0,31.0,44.0,39.0,39.0,29.0,15.0,24.0,41.0,30.0,23.0,25.0,40.0,26.0,26.0,27.0,40.0,25.0,31.0,31.0,21.0,33.0,20.0,22.0
3,バック・トゥ・ザ・フューチャー_ja.wikipedia_desktop_all-agents,232.0,306.0,340.0,318.0,271.0,292.0,396.0,386.0,341.0,374.0,358.0,329.0,316.0,346.0,341.0,428.0,365.0,377.0,341.0,324.0,320.0,392.0,412.0,286.0,353.0,349.0,326.0,366.0,358.0,418.0,363.0,365.0,384.0,383.0,377.0,361.0,367.0,355.0,321.0,...,467.0,417.0,442.0,425.0,394.0,475.0,344.0,528.0,399.0,522.0,435.0,408.0,378.0,428.0,401.0,414.0,404.0,370.0,431.0,349.0,489.0,492.0,417.0,359.0,420.0,436.0,418.0,482.0,423.0,1083.0,527.0,592.0,497.0,470.0,502.0,535.0,478.0,502.0,2007.0,1336.0
4,メーガン・マークル_ja.wikipedia_desktop_all-agents,199.0,128.0,197.0,164.0,200.0,200.0,603.0,327.0,301.0,215.0,234.0,1200.0,499.0,263.0,439.0,758.0,404.0,641.0,512.0,162.0,203.0,259.0,175.0,173.0,155.0,137.0,117.0,151.0,242.0,122.0,142.0,136.0,177.0,125.0,90.0,105.0,151.0,164.0,163.0,...,7.0,6.0,8.0,7.0,10.0,8.0,8.0,16.0,10.0,8.0,13.0,12.0,7.0,11.0,8.0,11.0,12.0,11.0,3.0,6.0,8.0,6.0,8.0,8.0,8.0,13.0,5.0,5.0,5.0,11.0,5.0,7.0,5.0,7.0,8.0,10.0,9.0,7.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63851,User:Kingchang711_zh.wikipedia_all-access_spider,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,4.0,0.0,1.0,NaN,1.0,11.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,2.0,1.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,2.0,NaN,0.0,...,NaN,1.0,3.0,1.0,NaN,2.0,NaN,NaN,1.0,0.0,0.0,1.0,3.0,1.0,3.0,NaN,3.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN,1.0,NaN,3.0,NaN,1.0,1.0,1.0,NaN,NaN,0.0,NaN,NaN,5.0,3.0,1.0,NaN
63852,User:Caoli